# Budget participatif : Implémentation de Rule X

- Alban Guerbois
- Atelier IA/Décision
- Référent: Jérôme Lang
- M2 MODO
- Université Paris Dauphine

Possibilités d'amélioration:
- Rendre le code compatible avec des utilités dans [0,1]
- Mieux définir les règles pour départager des projets en cas d'égalité
- Représenter les itérations de l'algorithme naïf

### Import de librairies

In [1]:
from sympy import symbols, solve
import pandas as pd
import numpy as np

In [2]:
class Project:

    def __init__(self, name, cost):
        self.name = name
        self.cost = int(cost)

In [3]:
class Voter:
    
    def __init__(self, choices):
        self.budget = 0
        self.choices = choices
        
    def has_chosen(self, project):
        return project in self.choices
    
    def set_budget(self, x):
        self.budget = x
        
    def diminish_budget(self, x):
        self.budget -= x

In [4]:
class ParticipatoryBudget:
    
    def __init__(self, projects, voters, budget):
        self.projects = projects
        self.voters = voters
        self.budget = budget
        self.n = len(voters)
        
    def add_voters(self, new_voters):
        self.voters.add(new_voters)
        
    def add_projects(self, new_projects):
        self.projects.add(new_projects)
       
        
    # fonction renvoyant les projets séléctionnés par la méthode rule X
    def ruleX(self):
        print('Rule X\n')
        k = 1
        selected = []
        candidates = self.projects.copy()
        # Chaque votant dispose d'une fraction du budget
        for voter in self.voters:
            voter.set_budget(1/self.n)

        while len(candidates) != 0:
            print("----------------------")
            print("Itération " + str(k))
            print("----------------------")
            best_project = None
            rho_min = 1

            print("Budget des votants")
            for idx, voter in enumerate(self.voters):
                print("Votant " + str(idx + 1) + ": " + str(voter.budget))
            print("\nCalcul du rho pour chaque projet candidat")
            
            for project in candidates.copy():
                rho = self.compute_rho(project)
                print(project.name + ": " + str(rho))
                if rho > 1/self.n:
                    candidates.remove(project)
                else:
                    if rho <= rho_min:
                        rho_min = rho
                        best_project = project

            if best_project:
                print("\nMEILLEUR PROJET: " + best_project.name + '\n')
                selected.append(best_project)
                self.update_budgets(rho_min, best_project)
                candidates.remove(best_project)           
            k += 1
            
        return selected
    
    # L'algorithme naïf séléctionne tant que possible, le projet avec le plus de votes
    # et le retient si le budget restant est suffisant
    def naive_algo(self):
        selected = []
        candidates = self.projects.copy()
        nb_votes = dict()
           
        for project in self.projects:
            nb_votes[project] = 0
            for voter in self.voters:
                if voter.has_chosen(project):
                    nb_votes[project] += 1
        
           
        budget_remaining = self.budget
        
        for project in sorted(nb_votes, key = nb_votes.get, reverse = True):
            if budget_remaining >= project.cost:
                selected.append(project)
                budget_remaining -= project.cost
                
        return selected
        
    # Calcule en O(n) le rho d'un projet pour que ce dernier soit rho-abordable 
    # On calcule entre quels points de discontinuité (breakpoints) la droite y = cout/budget vient couper la fonction affine par morceau
    # On récupère ensuite rho en resolvant une équation du premier degré 
    def compute_rho(self, project):
        k = 0
        breakpoints = [1/self.n]
        
        for voter in self.voters:
            if voter.has_chosen(project):
                k += 1
                breakpoints.append(voter.budget)
        y = 0
        previous = 0
        found = False
        for breakpoint in sorted(breakpoints):
            if project.cost/self.budget <= (k*(breakpoint-previous) + y):
                found = True
                break
            else:
                y = (k*(breakpoint-previous) + y)
                k -= 1
                previous = breakpoint
        if found == True:
            x = symbols('x')
            expr = k*x + y - project.cost/self.budget
            rho = solve(expr,x)[0] + previous
        else: 
            # Cas ou le rho excède le budget de 1/n
            rho = 1
        return rho
    
    # Met à jour les budgets des personnes ayant votés pour un certain projet après que celui-ci ait été selectionné
    def update_budgets(self, rho_min, best_project):
        for voter in self.voters:
            if voter.has_chosen(best_project):
                voter.diminish_budget(min(voter.budget, rho_min))
                
    def compare_ruleX_naive(self):
        names = [project.name for project in projects]
        res1 = self.ruleX()
        res2 = self.naive_algo()
        x1 = ['x' if project in res1 else '' for project in projects]
        x2 = ['x' if project in res2 else '' for project in projects]
        d = {'ruleX': x1, 'naive algo': x2}
        df = pd.DataFrame(data = d)
        df.index = names
        df.style.set_properties(**{'text-align': 'center'})
        return df
    
    def show_votes_matrix(self):
        names = [project.name for project in projects]
        cost = [project.cost for project in projects]
        d = [] 
        d.append(cost)
        for voter in voters:
            d.append([1 if project in voter.choices else 0 for project in projects])
        
        df = pd.DataFrame(data = np.transpose(d))
        columns = ['cost']
        for i in range(self.n):
            columns.append(i + 1)
        df.columns = columns
        df.index = names
        df['nb_votes'] = df.drop('cost', axis = 1).sum(axis = 1)
        df.iloc[:,1:self.n + 1] = df.iloc[:,1:self.n + 1].applymap(lambda x: 'x' if x == 1 else '')
        return df

# Exemple fictif

In [5]:
a = Project('Végétalisation facade', 3000)
b = Project('Achat piano libre service', 3500)
c = Project('4 Tables pique nique cour', 2000)
d = Project('Installation boite à livres', 1000)
e = Project('Matériel loisir en location', 1200)
f = Project('Billard/Baby foot', 1000)
g = Project('Encarts affichage libres couloirs', 800)

projects = [a, b, c, d, e, f, g]
voters = []
voters.append(Voter([a, b, e]))
voters.append(Voter([a, b, g]))
voters.append(Voter([a, b, g]))
voters.append(Voter([a, d, g]))
voters.append(Voter([a, e, f]))
voters.append(Voter([c, d, f]))
voters.append(Voter([c, e, f]))
voters.append(Voter([a, c, d]))
voters.append(Voter([e, f]))
voters.append(Voter([f, g]))

pb = ParticipatoryBudget(projects, voters, 10000)

## Matrice des votes

In [6]:
pb.show_votes_matrix()

,cost,1,2,3,4,5,6,7,8,9,10,nb_votes
Végétalisation facade,3000,x,x,x,x,x,,,x,,,6
Achat piano libre service,3500,x,x,x,,,,,,,,3
4 Tables pique nique cour,2000,,,,,,x,x,x,,,3
Installation boite à livres,1000,,,,x,,x,,x,,,3
Matériel loisir en location,1200,x,,,,x,,x,,x,,4
Billard/Baby foot,1000,,,,,x,x,x,,x,x,5
Encarts affichage libres couloirs,800,,x,x,x,,,,,,x,4


## Résultats par l'algo. naïf et par Rule X

In [7]:
pb.compare_ruleX_naive()

Rule X

----------------------
Itération 1
----------------------
Budget des votants
Votant 1: 0.1
Votant 2: 0.1
Votant 3: 0.1
Votant 4: 0.1
Votant 5: 0.1
Votant 6: 0.1
Votant 7: 0.1
Votant 8: 0.1
Votant 9: 0.1
Votant 10: 0.1

Calcul du rho pour chaque projet candidat
Végétalisation facade: 0.0500000000000000
Achat piano libre service: 1
4 Tables pique nique cour: 0.0666666666666667
Installation boite à livres: 0.0333333333333333
Matériel loisir en location: 0.0300000000000000
Billard/Baby foot: 0.0200000000000000
Encarts affichage libres couloirs: 0.0200000000000000

MEILLEUR PROJET: Encarts affichage libres couloirs

----------------------
Itération 2
----------------------
Budget des votants
Votant 1: 0.1
Votant 2: 0.0800000000000000
Votant 3: 0.0800000000000000
Votant 4: 0.0800000000000000
Votant 5: 0.1
Votant 6: 0.1
Votant 7: 0.1
Votant 8: 0.1
Votant 9: 0.1
Votant 10: 0.0800000000000000

Calcul du rho pour chaque projet candidat
Végétalisation facade: 0.0500000000000000
4 Tables p

,naive algo,ruleX
Végétalisation facade,x,x
Achat piano libre service,x,
4 Tables pique nique cour,,
Installation boite à livres,,x
Matériel loisir en location,x,x
Billard/Baby foot,x,x
Encarts affichage libres couloirs,x,x
